### Exercise - 4 Optimizers

In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
import numpy as np
from torch.utils.data import DataLoader
import torch.utils.data
import torch.nn.functional as F
import torchvision.transforms as transforms

from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d

from torch.utils.tensorboard import SummaryWriter
torch.set_printoptions(linewidth=120)

In [34]:
# Hyperparameters
num_epochs = 6
num_classes = 10
batch_size = 100
learning_rate = 0.001

DATA_PATH = '/home/jaishree/Documents/DA Sem 1/DDA Lab/CNN_image_classification/cifar-10'
MODEL_PATH = '/home/jaishree/Documents/DA Sem 1/DDA Lab/CNN_image_classification/results'

In [35]:
train_dataset = CIFAR10(root = DATA_PATH, download=True, transform=ToTensor())
test_dataset = CIFAR10(root = DATA_PATH, train=False, transform=ToTensor())

Files already downloaded and verified


In [36]:
#Data Augmentation: the process of artificially ’increasing’ our dataset by adding translation, scaling 
#and flipping to the images to fabricate examples for training.

train_len = 25*1000
part_train = torch.utils.data.random_split(train_dataset, [train_len, len(train_dataset)-train_len])[0]


train_loader = DataLoader(dataset=part_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [37]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU())
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc_layer = nn.Sequential(
            nn.Linear(7*7*128, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True)
        )
        
            
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)       
        out = out.reshape(out.size(0), -1)
        out = self.fc_layer(out)
        
        return out

In [9]:
#model = ConvNet()

# Loss and opitimizer
#criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [33]:

param_values = [0.001, 0.01, 0.1]

In [32]:
# Train the model
# Adam Optimizer

for l_rate in param_values:
    
    model = ConvNet()
    
    train_loader = torch.utils.data.DataLoader(part_train,batch_size = batch_size, shuffle = True)
    optimizer = torch.optim.Adam(model.parameters(), lr=l_rate)
    criterion = torch.nn.CrossEntropyLoss()
    
    total_step = len(train_loader)
    #train_loss = []
    #train_acc = []

    tb = SummaryWriter()

    for epoch in range(num_epochs):
        total_loss = 0
        total_correct = 0
        for i, (images, labels) in enumerate(train_loader):

            # Run the forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            #train_loss.append(loss.item())
            total_loss += loss.item()

            # Backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            total_correct += (predicted == labels).sum().item()
            #train_acc.append(correct / total)

        tb.add_scalar("Adam1 Train Loss", total_loss/len(part_train), epoch)
        tb.add_scalar("Adam1 Train Accuracy", total_correct/len(part_train), epoch)

        print('Epoch [{}/{}], Step [{}/{}], training Loss: {:.4f}, training Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, total_loss/len(part_train),
                          (total_correct / len(part_train)) * 100))
        
        tb.add_hparams(
            {"lr": l_rate},
            {
                "accuracy": total_correct/ len(part_train),
                "loss": total_loss/len(part_train),
            },
        )


    tb.close()
    
    ## testing the model
    
    model.eval()


    int_correct = 0
    int_total = 0
    int_total_loss = 0

    tb = SummaryWriter()

    with torch.no_grad():
        for i, (int_images, int_labels) in enumerate(test_loader):
            int_outputs = model(int_images)
            int_loss = criterion(int_outputs, int_labels)
            test_int_loss.append(int_loss.item())
            int_total_loss += int_loss.item()

            _, predicted = torch.max(int_outputs.data, 1)
            int_total += int_labels.size(0)
            int_correct += (predicted == int_labels).sum().item()
            correct = (predicted == labels).sum().item()
            test_int_acc.append(correct / int_labels.size(0))

        tb.add_scalar("Adam1 Test Loss", int_total_loss/len(test_dataset), l_rate)
        tb.add_scalar("Adam1 Test Accuracy", int_correct / int_total, l_rate)

        print('LR: {}, Test Accuracy of the model on all test images: {} %'.format(l_rate, (int_correct / int_total) * 100))


    tb.close()


Epoch [1/6], Step [250/250], training Loss: 0.0351, training Accuracy: 17.59%
Epoch [2/6], Step [250/250], training Loss: 0.0298, training Accuracy: 32.54%
Epoch [3/6], Step [250/250], training Loss: 0.0243, training Accuracy: 42.88%
Epoch [4/6], Step [250/250], training Loss: 0.0225, training Accuracy: 48.75%
Epoch [5/6], Step [250/250], training Loss: 0.0211, training Accuracy: 52.76%
Epoch [6/6], Step [250/250], training Loss: 0.0198, training Accuracy: 56.17%
LR: 54.71, Test Accuracy of the model on all test images: 0.001 %
Epoch [1/6], Step [250/250], training Loss: 0.0420, training Accuracy: 9.73%
Epoch [2/6], Step [250/250], training Loss: 0.0330, training Accuracy: 10.13%
Epoch [3/6], Step [250/250], training Loss: 0.0327, training Accuracy: 10.24%
Epoch [4/6], Step [250/250], training Loss: 0.0327, training Accuracy: 10.28%
Epoch [5/6], Step [250/250], training Loss: 0.0327, training Accuracy: 10.11%
Epoch [6/6], Step [250/250], training Loss: 0.0327, training Accuracy: 10.11%

In [38]:
# Train the model

# SGD Optimizer

for l_rate in param_values:
    
    model = ConvNet()
    
    train_loader = torch.utils.data.DataLoader(part_train,batch_size = batch_size, shuffle = True)
    optimizer = torch.optim.SGD(model.parameters(), lr=l_rate, momentum=0.9)
    criterion = torch.nn.CrossEntropyLoss()
    
    total_step = len(train_loader)
    #train_loss = []
    #train_acc = []

    tb = SummaryWriter()

    for epoch in range(num_epochs):
        total_loss = 0
        total_correct = 0
        for i, (images, labels) in enumerate(train_loader):

            # Run the forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            #train_loss.append(loss.item())
            total_loss += loss.item()

            # Backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            total_correct += (predicted == labels).sum().item()
            #train_acc.append(correct / total)

        tb.add_scalar("SGD Train Loss", total_loss/len(part_train), epoch)
        tb.add_scalar("SGD Train Accuracy", total_correct/len(part_train), epoch)

        print('Epoch [{}/{}], Step [{}/{}], training Loss: {:.4f}, training Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, total_loss/len(part_train),
                          (total_correct / len(part_train)) * 100))
        
        tb.add_hparams(
            {"lr": l_rate},
            {
                "accuracy": total_correct/ len(part_train),
                "loss": total_loss/len(part_train),
            },
        )


    tb.close()
    
    
    ## test step
    model.eval()


    int_correct = 0
    int_total = 0
    int_total_loss = 0

    tb = SummaryWriter()

    with torch.no_grad():
        for i, (int_images, int_labels) in enumerate(test_loader):
            int_outputs = model(int_images)
            int_loss = criterion(int_outputs, int_labels)
            test_int_loss.append(int_loss.item())
            int_total_loss += int_loss.item()

            _, predicted = torch.max(int_outputs.data, 1)
            int_total += int_labels.size(0)
            int_correct += (predicted == int_labels).sum().item()
            correct = (predicted == labels).sum().item()
            test_int_acc.append(correct / int_labels.size(0))

        tb.add_scalar("SGD Test Loss", int_total_loss/len(test_dataset), l_rate)
        tb.add_scalar("SGD Test Accuracy", int_correct / int_total, l_rate)

        print('LR: {}, Test Accuracy of the model on all test images: {} %'.format(l_rate, (int_correct / int_total) * 100))


    tb.close()

Epoch [1/6], Step [250/250], training Loss: 0.0474, training Accuracy: 9.67%
Epoch [2/6], Step [250/250], training Loss: 0.0371, training Accuracy: 13.75%
Epoch [3/6], Step [250/250], training Loss: 0.0359, training Accuracy: 15.69%
Epoch [4/6], Step [250/250], training Loss: 0.0357, training Accuracy: 16.52%
Epoch [5/6], Step [250/250], training Loss: 0.0353, training Accuracy: 19.19%
Epoch [6/6], Step [250/250], training Loss: 0.0344, training Accuracy: 23.04%
LR: 0.001, Test Accuracy of the model on all test images: 23.580000000000002 %
Epoch [1/6], Step [250/250], training Loss: 0.0389, training Accuracy: 10.17%
Epoch [2/6], Step [250/250], training Loss: 0.0346, training Accuracy: 17.67%
Epoch [3/6], Step [250/250], training Loss: 0.0276, training Accuracy: 26.36%
Epoch [4/6], Step [250/250], training Loss: 0.0163, training Accuracy: 39.28%
Epoch [5/6], Step [250/250], training Loss: 0.0147, training Accuracy: 45.68%
Epoch [6/6], Step [250/250], training Loss: 0.0133, training Acc